In [1]:
import geopandas as gpd

In [2]:
# this is the joined_unique dataframe containing all accidents with the assigned strava segment
accidents_with_segment = gpd.read_parquet("../data/dataframes/accidents_with_segment.parquet")

accident_locations = gpd.read_parquet("../data/dataframes/accident_locations.parquet")
strava_segments = gpd.read_parquet("../data/dataframes/strava_segments.parquet")
strava_segments['coord_list'] = strava_segments.geometry.apply(lambda line: list(line.coords))

In [3]:
from collections import defaultdict

# 1. Map every coordinate to the segments that pass through it
coords_to_segments = defaultdict(set)

for _, row in strava_segments.iterrows():
    name = row["counter_name"]
    # Using a set to avoid counting the same segment more than once
    for coord in row.coord_list:
        coords_to_segments[coord].add(name)

# 2. Filter the dictionary for "crossings" (more than 1 unique segment)
all_crossings = {
    coord: segments 
    for coord, segments in coords_to_segments.items()
    if len(segments) > 1
}

In [4]:
import folium; import numpy as np; import random

def plot_segments_accidents(crossing_dict, segment_df, joined_df, accident_locations_df,
                            coordinates, lat_tol, lon_tol, zoom, save_path=None, save=False):

    m = folium.Map(location=coordinates, zoom_start=zoom, tiles=None)
    folium.TileLayer(
    tiles='https://basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}.png',
    attr='©CartoDB',
    name='CartoDB Light No Labels').add_to(m)
    
    colors = ["red", "blue", "green", "orange", "purple", "pink", "yellow"]

    
    # 1. get all segments in the specified area
    # 2. add segment line for segment and correponding accidents in same color  
    
    min_lat, max_lat = coordinates[0] - lat_tol, coordinates[0] + lat_tol
    min_lon, max_lon = coordinates[1] - lon_tol, coordinates[1] + lon_tol
    sliced_df = segment_df[segment_df["latitude"].between(min_lat, max_lat) &
                segment_df["longitude"].between(min_lon, max_lon)]
    for segment_idx, segment_row in sliced_df.iterrows():
        color = random.choice(colors)
        segment_coords = [(lat, lon) for lon, lat in segment_row.geometry.coords]
        folium.PolyLine(
            locations=segment_coords,
            color=color,
            weight=5,
            opacity=0.5).add_to(m)
        # add all coordinates that make up a segment to the map
        # for point in segment_coords:
        #     folium.CircleMarker(
        #         location=point,
        #         radius=5,
        #         color=color,
        #         fill=True,
        #         fill_color=color,
        #         popup=f"Lat/Lon: {point}",
        #         fill_opacity=0.5,
        #         opacity=0.5,
        #     ).add_to(m)
        # plot accidents that belong to segment in the same color as the segment
        # for _, row in joined_df[joined_df["index_right"] == float(segment_idx)].iterrows():
        #     lat = row['YGCSWGS84']; lat_list.append(lat)
        #     lon = row['XGCSWGS84']; lon_list.append(lon)
        #     folium.CircleMarker(
        #         location=[lat, lon],
        #         radius=1,
        #         color=color,
        #         fill=True,
        #         fill_color=color,
        #         fill_opacity=1.0).add_to(m)
    
    # plot all accidents that are not added to any segment in black
    # mask = (~accident_locations_df['YGCSWGS84'].isin(lat_list) & accident_locations_df['YGCSWGS84'].between(min_lat, max_lat)) & (~accident_locations_df['XGCSWGS84'].isin(lon_list) & accident_locations_df['XGCSWGS84'].between(min_lon, max_lon))
    # filtered_rows = accident_locations_df[mask]
    # for _, row in filtered_rows.iterrows():
    #     lat = row['YGCSWGS84']
    #     lon = row['XGCSWGS84']
    #     folium.CircleMarker(
    #         location=[lat, lon],
    #         radius=1,
    #         color="black",
    #         fill=True,
    #         fill_color=color,
    #         fill_opacity=1.0).add_to(m)
        
    # plot all crossings as big black points
    for lon, lat in crossing_dict.keys():
        #if (min_lon < lon < max_lon) & (min_lat < lat < max_lat):
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            color="black",
            fill=True,
            fill_color="black",
            fill_opacity=1.0).add_to(m)

    if save:
        m.save(save_path)
        print(f"Map saved to {save_path}")
    return m

# --- using the function ---
coordinates = (52.518589, 13.376665) # coordinates for berlin mitte
lat_tol = 1.5e-2; lon_tol = 3e-2
zoom = 14
map_obj = plot_segments_accidents(all_crossings, strava_segments, accidents_with_segment, accident_locations,
                                  coordinates, lat_tol, lon_tol, zoom, save=False)
map_obj